<a href="https://colab.research.google.com/github/JacquelineBashta/Pandas_Eniac/blob/main/Project_2_Eniac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# orders.csv (ord_)
Every row in this file represents an order.
- order_id – a unique identifier for each order
- created_date – a timestamp for when the order was created
- total_paid – the total amount paid by the customer for this order, in euros
- state 
  - “Shopping basket” – products have been placed in the shopping basket, but the order has not been processed yet.
  - “Pending” – the shopping basket has been processed, but payment confirmation is pending.
  - “Completed” – the order has been placed and paid, and the transaction is completed.
  - “Cancelled” – the order has been cancelled and the payment returned to the customer.

    ==============================================================================================

## Importing the data
- ``` glob-glob("file_pat") ``` --> read multi files 
- ``` pd.concat(dfs_list, ignore_index=True)```  --> create 1 df from multi dfs
- ``` pd.read_csv(path)```  --> create 1 df from a csv file

In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.min_rows", 0) 
pd.set_option("display.max_rows", 30) 
pd.__version__

'1.4.4'

In [3]:
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing" 
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
ord = pd.read_csv(path)

      ===============================================

## Rename Columns , Set Index
- Rules: 
    - add 2-3 chars prefix to all columns
    - change column name to CamelCase
    - shorten it as max as possible
    - the Unique column shall have ALL_CAP name
 - ``` df.columns```   , ``` df.index``` 
 - ``` df=df.rename(columns={"A": "a", "B": "c"})``` 
 - ``` df.columns = ["a","b":"x"]``` 
     - take care, renaming the columns like that will convert the NAN to some value!!
 - ``` df=df.set_index("col")```  , ``` df=df.reset_index()``` 

In [4]:
ord.columns 

Index(['order_id', 'created_date', 'total_paid', 'state'], dtype='object')

In [5]:
ord=ord.rename(columns={"order_id": "ORD_ID"
                            , "created_date": "ord_CreatDate"
                            , "total_paid": "ord_TotlPaid"
                            , "state": "ord_State"})

In [6]:
ord.index 

RangeIndex(start=0, stop=226909, step=1)

    ==============================================================================================

## Explore the data
- ``` df.shape``` , ``` df.size``` , ``` df.ndim``` 
- ``` df.sample(5)``` , ``` df.info()``` 
- Numerical : ``` df.describe()``` , ``` df.col.nlargest()``` , ``` df.col.nsmallest()``` 
- Category : ``` df.nunique()``` , ``` df.unique() ``` 

In [7]:
ord.shape

(226909, 4)

In [8]:
ord.sample(5)

,ORD_ID,ord_CreatDate,ord_TotlPaid,ord_State
176102,476492,2017-12-29 16:50:17,23.74,Shopping Basket
92631,392210,2017-08-24 04:23:41,2713.59,Completed
194877,495360,2018-01-21 10:40:50,61.00,Shopping Basket
51316,350811,2017-04-28 17:15:43,42.99,Shopping Basket
216200,516691,2018-02-24 00:17:06,1099.00,Shopping Basket


In [9]:
ord.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ORD_ID         226909 non-null  int64  
 1   ord_CreatDate  226909 non-null  object 
 2   ord_TotlPaid   226904 non-null  float64
 3   ord_State      226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [10]:
ord.describe()

,ORD_ID,ord_TotlPaid
count,226909.000000,226904.000000
mean,413296.482480,569.225818
std,65919.250331,1761.778002
min,241319.000000,0.000000
25%,356263.000000,34.190000
50%,413040.000000,112.990000
75%,470553.000000,525.980000
max,527401.000000,214747.530000


In [11]:
ord.nunique() # hint: order_id is unique per row 
# hint: state is category data

ORD_ID           226909
ord_CreatDate    224828
ord_TotlPaid      31236
ord_State             5
dtype: int64

In [12]:
ord.ord_State.unique()
#hint: state has value "Shopping Basket"/"Pending" which ideally shall not be part of the analysis - TODO: fix required
# (if we care about actuall sold products)


array(['Cancelled', 'Completed', 'Pending', 'Shopping Basket',
       'Place Order'], dtype=object)

    ==============================================================================================

## Initial Clean
- Remember to create a copy of the df using ``` df.copy()``` ``` 

In [13]:
#keep original
ord_original = ord.copy()

      ===============================================

### Strip whitespaces
- ``` df.applymap(lambda x: x.strip() if isinstance(x, str) else x)```

In [14]:
ord = ord.applymap(lambda x: x.strip() if isinstance(x, str) else x)

      ===============================================
### Remove Duplicates Rows
- ``` df.duplicated().sum()``` 
- ``` df.loc[df.duplicated()==True]``` 
- ``` df=df.drop_duplicates() ``` 
- ``` df=df.drop_duplicates(subset=["col"])```  --> remove rows based on duplicated in specific column

In [15]:
ord.duplicated().sum() #hint : No duplicates

0

      =======================

#### Remove duplicated rows related to Unique columns
- find possible duplicates ``` len(df.ol_ID.unique()) == df.shape[0] ``` 
- get the excat value for duplicate columns ``` df.loc[df.duplicated(subset="col")]``` 
- find all columns with same value ``` df[df.col=="val"]``` 

In [16]:
len(ord.ORD_ID.unique()) == ord.shape[0]

True

      ===============================================

### Drop duplicate/un-necessary Columns
- ``` df=df.drop(columns=["col1","col2"])``` 

In [17]:
ord.sample(5) ## hint: all columns contain needed data. nothing to drop

,ORD_ID,ord_CreatDate,ord_TotlPaid,ord_State
195838,496321,2018-01-22 19:53:39,10.99,Shopping Basket
177771,478163,2017-12-31 17:30:55,129.20,Pending
212956,513447,2018-02-17 19:26:59,486.89,Completed
32746,332230,2017-03-05 20:29:19,0.00,Place Order
33373,332857,2017-03-07 10:26:43,45.98,Pending


In [18]:
ord.compare(ord_original)

Empty DataFrame
Columns: []
Index: []

      ===============================================

### Focus : Bussines case --> filter un-necessary Data
- ``` df=df.drop(columns=["col1","col2"])``` 

For the discount effect, no need for any order that with state != Completed

In [21]:
ord.shape

(226909, 4)

In [23]:

ord_comp=[]
ord_comp = ord.loc[ord.ord_State == "Completed"]

In [26]:
print(f"percentage of completed roders is {round((ord_comp.shape[0]/ord.shape[0])*100,2)}%")

percentage of completed roders is 20.54%


    ==============================================================================================

## Export the cleaned DataFrame

``` df.to_pickle("file_name.pkl")``` 

In [27]:
ord_comp.to_pickle("clean_Tables\Orders_c.pkl")